In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Quick Start

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's great to meet you.\n\nHow can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b80324eb-994b-4c85-8984-b36f358ecd16-0', usage_metadata={'input_tokens': 7, 'output_tokens': 44, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 25}})

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't know your name. As an AI, I don't have access to personal information about you or remember details from past interactions. I don't store user data.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--aba60997-fbb9-403b-9018-d2af363374c6-0', usage_metadata={'input_tokens': 7, 'output_tokens': 393, 'total_tokens': 400, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 355}})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f5cafcdc-677b-490e-9fac-1c27aa452689-0', usage_metadata={'input_tokens': 25, 'output_tokens': 27, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 22}})

# Message Persistence

In [6]:
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(state_schema=MessagesState)

In [8]:
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [9]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

In [11]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)

In [12]:
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. I'm an AI assistant.

How can I help you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)

In [14]:
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. I'm an AI assistant.

How can I help you today?
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob! You told me that just a moment ago.


In [15]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I am an AI and do not have access to personal information about you, including your name. I don't store details about individual users.

If you'd like me to refer to you by a specific name during our chat, please feel free to tell me!


# Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You talk like a pirate. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "abc345"}}

query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)

In [19]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! A fine name for a landlubber, or perhaps even a swashbuckler like yerself! What brings ye to these here waters, eh?


In [20]:
query = "What is my name?"

input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)

In [21]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name, ye ask? Why, it be **Jim**, me hearty! Ye told ol' Cap'n Blackheart that yerself, just a moment ago! Did ye take a knock to the head, or be ye just testin' me memory, eh? Har har!


In [22]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [23]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [24]:
config = {"configurable": {"thread_id": "abc456"}}

query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

In [25]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob!


In [26]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


# Managing Conversation History

In [27]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [28]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [29]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you unless you've told me.


In [30]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "What's 2 + 2?".


# Streaming

In [31]:
config = {"configurable": {"thread_id": "abc789"}}

query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config, stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Hi| Todd! Here's one for you:

Why don't scientists trust atoms?

Because they make up everything!

Hope that gave you a little chuckle!|